In [11]:
import networkx as nx
import pandas as pd

data = pd.read_csv('C:/Users/timothy/Downloads/Camden_Licensing_Applications_Beta.csv')
data = data[data['Application Type']=='Gaming Notification']
full_graph = nx.Graph()

full_graph.add_nodes_from(data['Application Reference'].unique(),color='black')
full_graph.add_nodes_from(data['Premises Name'].unique(),color='blue')
full_graph.add_nodes_from(data['Application Legislation'].unique(),color='green')
full_graph.add_nodes_from(data['Ward Name'].unique(),color='red')
full_graph.add_nodes_from(data['Application Status'].unique(),color='yellow')
full_graph.add_nodes_from(data['Application Type'].unique(),color='purple')
"""
col index to name
Application Reference	Application Type	Premises Name	Premises Address	Application Legislation	Application Status	Application Objections	Approved Date	Date Of Events	Expiry Date	Hearing Date	Last Date For Representations	Received Date	Refused Date	Cumulative Impact Policy Area Name	Ward Code	Ward Name	Easting	Northing	Longitude	Latitude	Spatial Accuracy	Last Uploaded	Location	Socrata ID	Organisation URI
0	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25
"""


#adding edges
for row in data.iterrows():
    full_graph.add_edge(row[1][2],row[1][0],name='made application',received=row[1][12])
    full_graph.add_edge(row[1][0],row[1][16],name='application has ward')
    full_graph.add_edge(row[1][16],row[1][2],name='Deals with')
    full_graph.add_edge(row[1][0],row[1][5],name='has status',approved=row[1][7],refused=row[1][13])
    full_graph.add_edge(row[1][0],row[1][4],name='is of legislation type')
    full_graph.add_edge(row[1][0],row[1][1],name='is of type')
    


In [12]:
import numpy as np
def distance(long1, lat1,long2,lat2):
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    long1 = np.radians(long1)
    long2 = np.radians(long2)
    #earth radius
    R = 6371
    a = (np.sin((lat2-lat1)/2))**2 + np.cos(lat1) * np.cos(lat2) * (np.sin((long2-long1)/2))**2
    c = 2 * np.arctan2(np.sqrt(a),np.sqrt(1-a))
    d = R * c
    return d
   

In [13]:
count = 0
for row in data.iterrows():
    count +=1
    print(count/len(data['Application Reference']),end='\r', flush=True)
    for subrow in data.iterrows():
        if row[1][0]==subrow[1][0]:
            pass
        else:
            dis = distance(row[1][19],row[1][20],subrow[1][19],subrow[1][20])
            if dis <0.5:
                full_graph.add_edge(row[1][2],subrow[1][2],name='Is close to')
                

In [15]:
nx.write_edgelist(full_graph,'gaming_graph.csv',delimiter=',')

In [39]:
all_nodes = full_graph.nodes(data=True)

nodes_df = pd.DataFrame(all_nodes,columns=['Node','Attributes'])
nodes_df.to_csv('all_nodes.csv')